## torch.nn 패키지를 활용하여 network 구성
1. autograd -> define models and differentiate them  
2. nn.Module -> layers  
3. forward(input) -> returns the output

### 1. network 정의

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
# learnable parameter 수 출력

params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

# network 통과 예시
input = torch.randn(1, 1, 32, 32) # 32x32 data, nSamples x nChannels x Height x Width
out = net(input) # net에 입력
print(out) # 통과 결과 보여줌

# grad buffer 0으로 설정, 파라미터 초기화하기
net.zero_grad()
out.backward(torch.randn(1, 10))

10
torch.Size([6, 1, 3, 3])
tensor([[ 0.0645, -0.1434,  0.1670,  0.0097,  0.2549,  0.0241, -0.0118,  0.1047,
          0.0741, -0.0842]], grad_fn=<AddmmBackward>)


/storage/kjwoo31/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


### 2. loss 계산

In [3]:
# Loss function

output = net(input)

target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss() # Loss 함수 (MSE: mean squared error)

loss = criterion(output, target)
print(loss)

tensor(0.3224, grad_fn=<MseLossBackward>)


### 3. Backprop

In [4]:
# Backprop : loss.backward()를 통해 layer의 gradient update

net.zero_grad()     # 0으로 초기화

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0031, -0.0052,  0.0035, -0.0066,  0.0103,  0.0049])


### 4. update weight

In [5]:
# update weight
# SGD(Stochastic Gradient Descent) 사용: weight = weight - learning_rate * gradient

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [6]:
# SGD, Nesterov-SGD, Adam, RMSProp 등 다른 update 방법을 사용하려면 torch.optim 사용
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update